In [1]:
def get_input(fname="input.txt"):
    with open(fname) as f:
        return [int(n) for n in f.readline().strip().split(",")]

In [2]:
test_instructions = get_input("test.txt")

In [3]:
test_instructions[:5]

[109, 1, 204, -1, 1001]

In [4]:
test_instructions[-5:]

[101, 1006, 101, 0, 99]

In [5]:
import itertools

In [6]:
class Program(object):
    instructions = None
    input = None
    output = None
    is_halted = False
    idx = 0
    relative_base = 0
    
    STOPPED = -1
    NEEDS_INPUT = -2
    
    def __init__(self, instructions, input, output):
        self.instructions = {i: v for i, v in enumerate(instructions)}
        self.input = input
        self.output = output
    
    def _get_address(self, pos, mode=1):
        if mode == 0:
            return self.instructions.get(pos, 0)
        elif mode == 2:
            return self.instructions.get(pos, 0) + self.relative_base
    
    def _get_value(self, pos, mode=1):
        v = self.instructions.get(pos, 0)
        if mode == 0:
            v = self.instructions.get(v, 0)
        elif mode == 2:
            v = self.instructions.get(v + self.relative_base)
        return v
    
    def step(self):
        if self.is_halted:
            return Program.STOPPED
        opcode = self.instructions.get(self.idx, 0)
        op = opcode % 100
        opcode //= 100
        if op == 99: # halt
            self.is_halted = True
            return Program.STOPPED
        elif op == 9: # set relative base
            mode = opcode % 10
            v = self._get_value(self.idx + 1, mode)
            self.relative_base += v
            self.idx += 2
        elif op in (1, 2, 7, 8): # addition, multiplication, less than, equals
            modes = []
            for i in range(3):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b, idx_o = self.idx + 1, self.idx + 2, self.idx + 3
            mode_a, mode_b, mode_o = modes
            a = self._get_value(idx_a, mode_a)
            b = self._get_value(idx_b, mode_b)
            o = self._get_address(idx_o, mode_o)
            if op in (1, 2): # addition, multiplication
                res = a + b if op == 1 else a * b
            elif op == 7:
                res = int(a < b)
            elif op == 8:
                res = int(a == b)
            self.instructions[o] = res
            self.idx += 4
        elif op in (5, 6): # jump if true / false
            modes = []
            for i in range(2):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b = self.idx + 1, self.idx + 2
            mode_a, mode_b = modes
            a = self._get_value(idx_a, mode_a)
            b = self._get_value(idx_b, mode_b)
            if op == 5:
                if a != 0:
                    self.idx = b
                else:
                    self.idx += 3
            elif op == 6:
                if a == 0:
                    self.idx = b
                else:
                    self.idx += 3
        elif op == 3: # read input
            if len(self.input) == 0:
                return Program.NEEDS_INPUT
            mode = opcode % 10
            self.instructions[self._get_address(self.idx + 1, mode)] = self.input.popleft()
            self.idx += 2
        elif op == 4: # write output
            mode = opcode % 10
            output = self._get_value(self.idx + 1, mode)
            self.output.append(output)
            self.idx += 2
    

In [7]:
from collections import deque

In [8]:
p = Program(test_instructions, deque(), deque())

In [9]:
while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass

In [10]:
p.output

deque([109,
       1,
       204,
       -1,
       1001,
       100,
       1,
       100,
       1008,
       100,
       16,
       101,
       1006,
       101,
       0,
       99])

In [11]:
p = Program(get_input("test2.txt"), deque(), deque())
while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass
p.output

deque([1219070632396864])

In [12]:
p = Program(get_input("test2.txt"), deque(), deque())
while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass
p.output

deque([1219070632396864])

In [13]:
p = Program(get_input(), deque([1]), deque())
while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass
p.output

deque([2890527621])

In [14]:
p = Program(get_input(), deque([2]), deque())
while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass
p.output

deque([66772])